In [ ]:
# Import magics

!pip install selenium
!pip install webdriver_manager


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import re
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

output = Path('ISDS/Exam/output')

## Scrapping KU


In [43]:
def extract_ku(fag):
    
    headers = {'User-Agent':'bwt973@alumni.ku.dk'}
    
    url = f"https://studier.ku.dk/kandidat/{fag}/faglig-profil-og-job/"
    
    r = requests.get(url, headers)

    soup = BeautifulSoup(r.content.decode("utf-8"), "html.parser")
    
    return soup

def transform_ku(soup):

    divs = soup.find_all("div", class_="col-xs-12 col-sm-8 col-md-6 main-content")

    text_ku =[]

    for i in range(len(divs)):
        text_ku.append(divs[i].find_all("p"))

    divs2 = soup.find_all("div", class_="col-xs-12 col-sm-8 col-md-6 main-content")

    for i in range(len(divs2)):
        text_ku.append(divs2[i].find_all("ul"))
    
    return text_ku

ku_list = []

search_list = ['psykologi', 'sociologi', 'statskundskab', 'antropologi', 'oekonomi']


for k in search_list:
    soup = extract_ku(k)
    ku_list.append(transform_ku(soup))

ku_df = pd.DataFrame(data=ku_list)

#Merging the two colums 
ku_df=ku_df[0]+ku_df[1]

#Making the object a dataframe
ku_df=pd.DataFrame(ku_df)

ku_df=pd.DataFrame.transpose(ku_df)

ku_df.columns=['cand.psych', 'cand.scient.soc', 'cand.scient.pol', 'cand.scient.anth', 'cand.oecon']

In [44]:
#ku_df['KU'].loc[4]
ku_df

,cand.psych,cand.scient.soc,cand.scient.pol,cand.scient.anth,cand.oecon
0,"[[Psykologi er studiet af menneskets tænkning,...","[[Som sociolog er du interesseret i, hvordan o...",[[Hvis du interesserer dig for samfundet omkri...,[[Som færdiguddannet antropolog kan du arbejde...,[[Din internetbrowser understøtter ikke iframe...


## Data Cleaning KU 

## Scrapping AAU

In [45]:
def extract_aau(fag):
    
    headers = {'User-Agent':'bwt973@alumni.ku.dk'}
    
    url = f"https://www.aau.dk/uddannelser/kandidat/{fag}"
    
    r = requests.get(url, headers)

    soup = BeautifulSoup(r.content.decode("utf-8"), "html.parser")
    
    return soup

def transform_aau(soup):

    divs = soup.find_all("main", class_="Main_Main__2KIvG")

    for item in divs:
        text_aau = item.find_all()[0].text.strip()

        aau_text = {
            "text_aau" : text_aau, 
        }
        aau_list.append(aau_text)

    return text_aau

aau_list = []

search_list = ['psykologi', 'sociologi', 'oekonomi']

for k in search_list:
    try: 
        soup = extract_aau(k)
        transform_aau(soup)
    except:
        break

aau_df = pd.DataFrame(data=aau_list)

aau_df=pd.DataFrame.transpose(aau_df)

aau_df.columns=['cand.psych', 'cand.scient.soc', 'cand.oecon']

In [46]:
aau_df
#aau_df['text_aau'].loc[0]

,cand.psych,cand.scient.soc,cand.oecon
text_aau,KandidatPsykologiUddannelsen uddanner dig til ...,KandidatSociologiUddannelsen bygger videre på ...,KandidatØkonomiDu lærer at analysere samfundsø...
